In [2]:
from nltk.tokenize import TreebankWordTokenizer
from pymystem3 import Mystem
from nltk.tokenize import sent_tokenize
import copy
import pandas as pd
from collections import Counter
import nltk
import unicodedata
import re
import os
#fp = open('Fuji_ansi.txt')
#data = fp.read()


In [42]:
tokenizer = TreebankWordTokenizer()
path = os.getcwd()
print("current path:", path)
path = "C:/Users/Administrator/{}_chapt_data".format('FujiUTF.txt')
os.mkdir(path)

current path: C:\Users\Administrator


In [70]:
doc_tokens = []
for line in sent_split:    
    doc_tokens += [sorted(tokenizer.tokenize(line.lower()))]

In [114]:
import re
import os
from os import path

"""The function splits up .txt utf-8 file into separate files by chapters and saves them in corpus_data directory.
To call the function, put a file-path and re-pattern as arguments. Returns [TO DO] a list of file-paths of the chapters 
for processing by other functions"""
def chapter_splitter(file, pattern):

    if path.exists('C:\\Users\\administrator\\corpus_data\\') == True:
        pass
    elif path.exists('C:\\Users\\administrator\\corpus_data\\') == False:
        os.mkdir('corpus_data')
    
    
    pattern = re.compile(pattern)

    with open(file, 'r', encoding='utf-8') as fp:
        doc = fp.readlines()
        n = 0 #Chapter number counter
        for line in doc:
            if n == 0:
                output = open('corpus_data\intro.txt'.format(n), 'w', encoding='utf-8') #TO FIX INTRO!!
                output.write(line)
            matches = pattern.finditer(line)

            for match in matches:
                n += 1
                output = open('corpus_data\chapter_{}.txt'.format(n), 'w', encoding='utf-8')
                chapter_path = ('corpus_data\\chapter_{}.txt'.format(n))

            output.write(line)

        output.close()
        
        return os.fsdecode(chapter_path) #to make a list of filepaths
    
ch = chapter_splitter('Fuji.txt', r'\d\.\d.?\s')
print(ch)

<generator object chapter_splitter at 0x0000000019271350>


In [115]:
fp = open('FujiUTF.txt', encoding='utf-8')

def lemmatizer(text_path):
    sent_split = []
    for line in text_path:
        sent_split += sent_tokenize(line)
    sentences_copy = []
    for sentence in sent_split:
        sentences_copy += [copy.copy(sentence)]

    m = Mystem()
    lemmas =[]
    for sentence in sentences_copy:
        lemmas += [m.lemmatize(sentence)]
        lemmas = [''.join(words for words in sentence) for sentence in lemmas]
        print(round(len(lemmas)/len(sentences_copy)*100, 3))
        
    output = open('%lemmatizedUTF.txt'.format(text_path), 'w', encoding='utf-8')
    
    return output.write(''.join(lemmas))

    
corpus_ch = lemmatizer(fp)
fp.close()

0.011
0.022
0.033
0.044
0.055


KeyboardInterrupt: 

In [113]:
"""Splits text on sentences and saves them into a list without keeping chapters splitted. 
    Removes punctuation"""
def clean_sentences(text): #To make be able to take a list of filepaths
    fp = open(text, 'r', encoding='utf-8')
    data = fp.read()
    sent_split = sent_tokenize(data)
    puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';',
                 ', ', '. ', '! ', '? ', ': ', '""', '— ', '« ', '» ', ') ', '(', '; ',
            ' — ', ', — ', '; ', ' (', ' «', '-', '), ', ': «', '», ', ']',
            '[', '] ', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '–', '…'))
    stopwords = nltk.corpus.stopwords.words('russian')
    
    memo_lemmas_copy = []
    for sentence in sent_split:
        memo_lemmas_copy += [copy.copy(sentence)]
        
    memo_lemmas_copy = [(letter.replace('\n', ' ') for letter in sentence)
                       for sentence in memo_lemmas_copy]
    memo_lemmas_copy = [''.join(letter for letter in sentence if letter not in puncs)
                        for sentence in memo_lemmas_copy]

    df = pd.DataFrame(memo_lemmas_copy, columns=['text'])
    fp.close()
    return df

corpus = clean_sentences(ch)
corpus

TypeError: expected str, bytes or os.PathLike object, not generator

In [26]:
def bag_of_words(text_path, lemmatizer): #!!!ADAPT TO LEMMATIZER FUNC AND RAW TEXT!!!
    pass
    puncs = set((',', '.', '!', '?', ':', ':', '""', '—', '«', '»', ')', '(', ';',
            ', ', '. ', '! ', '? ', ': ', '""', '— ', '« ', '» ', ') ', '(', '; ',
            ' — ', ', — ', '; ', ' (', ' «', '-', '), ', ': «', '», ', ']',
            '[', '] ', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0', '–', '…'))
    stopwords = nltk.corpus.stopwords.words('russian')
    stopwords += set(('это', 'сказал', 'сказала', 'очень', 'просто', 'такое', 'что-то', 'тебе',
                 'вообще','сразу','несколько','почему','именно','самое','спросил','сказать','ответила'))
    all_doc_tokens = sum(doc_tokens, [])

    word_list = [x for x in all_doc_tokens 
             if x not in puncs 
             if x not in stopwords]

    lexicon = sorted(set(word_list))
    count_words = Counter(word_list)
    print('Объем лексикона (словоформы) без стопслов:', len(lexicon))
    print('Количество токенов без стопслов и знаков препинания:', len(word_list))
    #count_words.most_common(30)

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize.casual import casual_tokenize

tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=corpus.text).toarray()

tfidf_docs = pd.DataFrame(tfidf_docs)
tfidf_docs = tfidf_docs - tfidf_docs.mean()
tfidf_docs.shape

(484, 1652)

In [75]:
from sklearn.decomposition import PCA

pca = PCA(n_components=16)
pca = pca.fit(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
index = ['sentence{}'.format(i) for i in range(len(corpus))]
columns = ['topic{}'.format(i) for i in range(pca.n_components)]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=index)
#pca_topic_vectors.round(2).head(106)

In [76]:
column_nums, terms = zip(*sorted(zip(tfidf.vocabulary_.values(), tfidf.vocabulary_.keys())))
#terms

In [77]:
weights = pd.DataFrame(pca.components_, columns=terms, index=['topic{}'.format(i) for i in range(16)])
pd.options.display.max_columns = 8
#weights.head(10).round(3)

In [78]:
pd.options.display.max_columns = 12
deals = weights['душа тело женщина душа тяжесть'.split()].round(3)*100
#deals

In [79]:
#deals.T.sum()

In [80]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=16, n_iter=100)
svd_topic_vectors = svd.fit_transform(tfidf_docs.values)
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns, index=index)
#svd_topic_vectors.round(3).head(6)

In [45]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=corpus.text).toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(), counter.vocabulary_.keys())))
bow_docs.columns = terms

In [81]:
#corpus.loc[50].text

In [82]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA

ldia = LDiA(n_components=16, learning_method='batch')
ldia = ldia.fit(bow_docs)
#ldia.components_.shape

In [83]:
pd.set_option('display.width', 75)
components = pd.DataFrame(ldia.components_.T, index=terms, columns=columns)
#components.round(2).head(3)

In [84]:
#components.topic11.sort_values(ascending=False)[:60]